# Turning a Google Colab notebook into a web app

---

### This notebook is designed to be used alongside Anvil's [turning a Colab notebook into a web app tutorial](https://anvil.works/learn/tutorials/google-colab-to-web-app).

The text cells below tell you the steps you need to take to connect this notebook to an Anvil app. The steps are:


1. Install the `anvil-uplink` library
2. Import the `anvil.server` package
3. Connect the notebook using your apps Uplink key
4. Create a function to call from your app that includes the `anvil.server.callable` decorator
5. Add `anvil.server.wait_forever()` to the end of the notebook

### Follow along below for more detail.

---





### Let's start by installing the `anvil-uplink` library, all we need to do is add `!pip install anvil-uplink` to the first cell of the notebook:

In [7]:
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


### Next import the Anvil server package by adding `import anvil.server`:

Importing `anvil.server` means, when this notebook is connected via the Uplink, it will behave in the same way as any other [Anvil Server Module](https://anvil.works/docs/server).

In [8]:
import anvil.server

### Then connect this notebook to your app using your Uplink key `anvil.server.connect("your-uplink-key")`:

For information on how to get your apps Uplink key, see [Step 4 - Enable the Uplink](https://anvil.works/learn/tutorials/google-colab-to-web-app#step-4-enable-the-uplink).

In [9]:
anvil.server.connect("DAPAOJPOQFFRAFTAOLZBRB7K-YSRPAJWPBRTZT4FC")

### Build and train the classification model

The next cell gets the dataset, finds an optimal number of neighbours and then builds and trains the model. How this works is outside the scope of this tutorial, however, if you want to read more about how the code below works, Towards Data Science has a useful article [here](https://towardsdatascience.com/knn-using-scikit-learn-c6bed765be75).

#### *We don't need to change anything in the next cell.*


In [10]:
pip install requests beautifulsoup4 lxml 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
from typing import Text
from pandas.io.formats.format import TextAdjustment

In [12]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 985 kB 3.8 MB/s 
     |████████████████████████████████| 139 kB 43.0 MB/s 
     |████████████████████████████████| 358 kB 59.8 MB/s 
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 4.1 MB 37.0 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.11 which is incompatible.


In [13]:
"""convocatoria = 'ayuda economica'
clave = 'mujeres'"""

"convocatoria = 'ayuda economica'\nclave = 'mujeres'"

In [14]:
#Obtener datos búqueda en google

headers = {
"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
"Accept-Encoding":"gzip, deflate",
"Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
"DNT":"1"
}

df = pd.DataFrame(columns = ['palabra clave', 'titulo', 'enlace', 'descripcion'])

def obtener_resultados(termino_busqueda, numero_resultados, codigo_lenguaje):
  url_google = 'https://www.google.com/search?q={}&num={}&h1={}'.format(termino_busqueda, numero_resultados, codigo_lenguaje)
  respuesta = requests.get(url_google, headers = headers)
  respuesta.raise_for_status()
  return termino_busqueda, respuesta.text

def procesar_resultados(html, palabra):
  soup_response = soup(html, 'html.parser')
  bloque = soup_response.find_all("div", class_="g")
  global df  
  df = df.drop(df.index[range(0, df.shape[0])])
  
  for resultado in bloque:
    titulo = resultado.find('h3').get_text()
    enlaces = resultado.find('a',href=True).get('href')
    if resultado.find('div', {"class": "VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf"}) is not None:
        descripciones = resultado.find('div', {"class": "VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf"}).get_text()
    else:
      descripciones = 'Visita página web'

    df = df.append({'palabra clave' : palabra, 
                    'titulo' : titulo, 
                    'enlace' : enlaces, 
                    'descripcion' : descripciones},
                    ignore_index = True)   
  return df

def scrap(termino_busqueda, numero_resultados, codigo_lenguaje):
  palabra, html = obtener_resultados(termino_busqueda, numero_resultados, codigo_lenguaje)
  resultados = procesar_resultados(html, palabra)
  return resultados

In [15]:
# Creación de variables a partir de datos de búsqueda

#para extraer palabras clave
d = {'migrantes' : ['migrantes', 'migrante', 'migraciones'],
     'vejez' : ['vejez', 'personas mayores', 'ancianos'],
     'niñez': ["niñez", 'niños', 'niñas', 'infancia', 'menores', 'infantil', 'nino', 'nina', 'ninos', 'ninas'],
     'adolescentes': ['adolescentes', 'jovenes', 'jóvenes', 'adolescencia'],
     'mujeres' : ['mujeres', 'mujer', 'igualdad de género'],
     'salud': ['salud', 'enfermedades', 'enfermedad', 'médica', 'medica'],
     'educacion': ['educacion', 'educación', 'aprendizaje', 'becas', 'beca', 'estudiantes', 'estudiante', 'alfabetización', 'alfabetizacion', 'escolar', 'escuela', 'colegio', 'educar', 'educativo', 'alumano', 'alumnos'],
     'alimentacion': ['alimentacion', 'alimentación', 'alimentos', 'hambre', 'alimentario', 'comida', 'alimentaria', 'nutrición', 'nutricion'],
     'ambiental' : ['ambiental', 'medio ambiente'],
     'seguridad' : ['seguridad'],
     'comunitario' : ['comunicatorio', 'comunitaria'],
     'ciencia' : ['ciencia'],
     'tecnologia' : ['tecnologia', 'tecnología', 'innovacion', 'innovación'],
     'naciones unidas' : ['naciones unidas', 'onu'],
     'sexualidad' : ['sexualidad'],
     'salubridad' : ['salubridad', 'saneamiento'],
     'desastres' : ['desastres', 'desastre natural', 'desastre'],
     'discapacidad' : ['discapacidad', 'discapacitados', 'discapacitada', 'discapacitado'],
     'vulnerabilidad' : ['vulnerabilidad', 'vulnerables', 'vulnerable'],
     'pobreza' : ['pobreza', 'pobre', 'pobres', 'pago'],
     }

def mapper(val):
    for key, values in d.items():
        if any(value in val for value in values):
            return key
    return 'Otros'

def palabra_clave(resultados):
  resultados['clave'] = resultados['titulo'].map(mapper)

  if 'Otros' in resultados['clave']:
    resultados['clave'] = resultados['descripcion'].map(mapper)

  return resultados

# Para extraer pais
Pais = {'COLOMBIA' : ['colombia', '.co.', '.co/'],
        'ESPAÑA' : ['españa', '.es.', '.es/'],
        'ARGENTINA' : ['argentina', '.ar.', '.ar/'],
        'BOLIVIA' : ['bolivia', '.bo.', '.bo/'],
        'BRASIL' : ['brasil', 'brazil', '.br.', '.br/'],
        'CHILE' : ['chile', '.cl.', '.cl/'],
        'COSTA RICA' : ['costa rica', '.cr.', '.cr/'],
        'CUBA': ['cuba', '.cu.', '.cu/'],
        'ECUADOR' : ['ecuador', '.ec.', '.ec/'],
        'EL SALVADOR' : ['el salvador', '.sv.', '.sv/'],
        'ESTADOS UNIDOS' : ['estados unidos', '.us.', '.us/', '.usa.'],
        'GUATEMALA' : ['guatemala', '.gt.', '.gt/'],
        'GUYANA' : ['guyana', '.gf.', '.gf/'],
        'HAITI': ['haiti', 'haití', '.ht.', '.ht/'],
        'HONDURAS' : ['honduras', '.hn.', '.hn/'],
        'MEXICO' : ['mexico', 'méxico', '.mx.', '.mx/'],
        'NICARAGUA' : ['nicaragua', '.ni.', '.ni/'],
        'PANAMA' : ['panamá', 'panama', '.pa.', '.pa/'],
        'PARAGUAY' : ['paraguay', '.py.', '.py/'],
        'PERU' : ['perú', 'peru', '.pe.', '.pe/'],
        'REPUBLICA DOMINICANA' : ['república dominicana', 'republica dominicana', '.do,', '.do/'],
        'URUGUAY' : ['uruguay', '.uy.', '.uy/'],
        'VENEZUELA' : ['venezuela', '.ve.', '.ve/']
        }
    
def mapper1(val):
    for key, values in Pais.items():
        if any(value in val for value in values):
            return key
    return 'Por identificar'

def variable_pais(resultados):
  resultados['Pais'] = resultados.enlace.map(mapper1)

  return resultados

#Para extraer ODS
ODS = {'1' : ['pobreza', 'migrantes', 'vulnerabilidad', 'vulnerables', 'vulnerable', 'desastres', 'desastre natural', 'desastre', 'pobre', 'pobres',
              'pago', 'migrante', 'emigrante',  'migraciones', 'ayuda financiera'],
       '2' : ['alimentacion', 'alimentación', 'alimentos', 'hambre', 'alimentario', 'comida', 'alimentaria', 'nutrición', 'nutricion'],
       '3' : ['salud', 'enfermedades', 'enfermedad', 'médica', 'medica', 'sexualidad'],
       '4' : ['educacion', 'educación', 'aprendizaje', 'becas', 'beca', 'estudiantes', 'estudiante', 'alfabetización', 'alfabetizacion', 
              'escolar', 'escuela', 'colegio', 'educar', 'educativo', 'alumno', 'alumnos'],
       '5' : ['mujeres','mujer', 'igualdad de género'],
       '6' : ['salubridad', 'saneamiento'],
       '7' : ['energia'],
       '8' : ['trabajo'],
       '9' : ['ciencia', 'tecnologia', 'tecnología', 'innovacion', 'innovación', 'programación', 'tic'],
       '10' : ['discapacidad', 'discapacitados', 'discapacitada', 'discapacitado'],
       '11' : ['comunitario', 'sostenibilidad', 'seguridad', 'comunitaria'],
       '12' : ['produccion', 'consumo', 'producción'],
       '13' : ['ambiental', 'medio ambiente', 'clima'],
       '14' : ['mar', 'oceano', 'mares', 'peces'],
       '15' : ['ecosistema', 'ecosistemas'],
       '16' : ['paz', 'justicia', 'corrupción'],
       '17' : ['alianza', 'alianzas']
       }
    
def mapper2(val):
    for key, values in ODS.items():
        if any(value in val for value in values):
            return key
    return 'Por identificar'

def variable_ODS(resultados):
  resultados['ODS'] = resultados['titulo'].map(mapper2)

  if 'Por identificar' in resultados['ODS']:
    resultados['ODS'] = resultados['descripcion'].map(mapper2)
  
  return resultados


#Para extraer beneficiarios
Beneficiarios = {'niños y niñas' : ['niñez'],
                 'adulto mayor' : ['vejez'],
                 'jovenes' : ['adolescentes'],
                 'mujeres' : ['mujeres'],
                 'migrantes' : ['migrantes']
                 }

def mapper3(val):
    for key, values in Beneficiarios.items():
        if any(value in val for value in values):
            return key
    return 'mixto'

def variable_beneficiarios(resultados):
  resultados['Beneficiarios'] = resultados.clave.map(mapper3)

  if 'mixto' in resultados['Beneficiarios']:
    resultados['Beneficiarios'] = resultados['Beneficiarios'].map(mapper3)
  
  return resultados

### Next, we will create our `predict_iris()` function with a `@anvil.server.callable` decorator. The decorator makes the function callable from our Anvil app. 
Add the following code to the next cell:
```
@anvil.server.callable
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
  classification = knn.predict([[sepal_length, sepal_width, petal_length, petal_width]])
  return iris.target_names[classification][0]
```





In [16]:
@anvil.server.callable
def bdresultados(convocatoria, clave):
  convocatoria = convocatoria.split(',')
  clave = clave.split(',')
  palabra = []

  for i in convocatoria:
    for j in clave:
      concat = str(i+" + "+j)
      palabra.append(concat)

  resultados = pd.DataFrame(columns = ['palabra clave', 'titulo', 'enlace', 'descripcion'])  

  for i in palabra:
    resultados = pd.concat([resultados, scrap(i, 5, "es")])

  resultados = resultados.drop_duplicates(keep='last', subset="titulo")
  resultados = resultados.reset_index()
  resultados = resultados.drop(columns=['index'])
  resultados['titulo'] =  resultados['titulo'].str.lower()
  resultados['descripcion'] =  resultados['descripcion'].str.lower()

  resultados = palabra_clave(resultados)
  resultados = variable_pais(resultados)
  resultados = variable_ODS(resultados)
  resultados = variable_beneficiarios(resultados)

  bd = resultados.to_csv()
    
  return bd

### Finally, let's add `anvil.server.wait_forever()` function so the notebook is always available to the web app:

In [ ]:
anvil.server.wait_forever()

---

## That's it, 5 simple steps to connect your notebook to your Anvil app! 

---